In [40]:
import sys
import os
import re

current_directory = os.getcwd()
logfolder_path = os.path.join(current_directory, 'logfiles')
log_files = os.listdir(logfolder_path)
log_files = [file for file in log_files if file.endswith('.txt')]

log_files[0]

"HH20230401 T3570601767 No Limit Hold'em $0.92 + $0.08.txt"

In [41]:
len(log_files)

4344

In [115]:
output_file_path = os.path.join(current_directory, 'row_content_info.txt')

first_txt_file = log_files[0]
first_txt_file_path = os.path.join(logfolder_path, first_txt_file)

#count variables
game = 1
row_number = 0
hand = 0

#state variables
start_hand = 0
preflop = 0
flop = 0
turn = 0
river = 0
showdown = 0

#player variables
dealer_position = 0
seats = {}

pattern_start = r"PokerStars"
pattern_preflop = r"HOLE CARDS"
pattern_flop = r"FLOP"
pattern_turn = r"TURN"
pattern_river = r"RIVER"
pattern_showdown = r"SHOW DOWN"
pattern_summary = r"SUMMARY"
pattern_dealer_position = r"Seat #(\d+)"
pattern_player_information = r"Seat (\d+): ([a-zA-Z0-9]+) \((\d+) in chips\)"

with open(first_txt_file_path, 'r') as file, open(output_file_path, 'w') as output_file:
    for line in file:
        row_number += 1 
        has_content = bool(line.strip())  

        contains_start = bool(re.search(pattern_start, line))
        contains_holed_cards = bool(re.search(pattern_preflop, line))
        contains_flop = bool(re.search(pattern_flop, line))
        contains_turn = bool(re.search(pattern_turn, line))
        contains_river = bool(re.search(pattern_river, line))
        contains_showdown = bool(re.search(pattern_showdown, line))
        contains_summary = bool(re.search(pattern_summary, line))
        
        #start of a hand; first set state variables
        if contains_start == True:
            start_hand = 1
            preflop = 0
            flop = 0
            turn = 0
            river = 0
            showdown = 0
        
        if contains_holed_cards == True:
            start_hand = 0
            preflop = 1
            flop = 0
            turn = 0
            river = 0
            showdown = 0

        if contains_flop == True:
            start_hand = 0
            preflop = 0
            flop = 1
            turn = 0
            river = 0
            showdown = 0

        if contains_turn == True:
            start_hand = 0
            preflop = 0
            flop = 0
            turn = 1
            river = 0
            showdown = 0

        if contains_river == True:
            start_hand = 0
            preflop = 0
            flop = 0
            turn = 0
            river = 1
            showdown = 0

        if contains_showdown == True:
            start_hand = 0
            preflop = 0
            flop = 0
            turn = 0
            river = 0
            showdown = 1

        if contains_summary == True:
            start_hand = 0
            preflop = 0
            flop = 0
            turn = 0
            river = 0
            showdown = 0
            seats = {}

        if has_content == False:
            row_number = 0

        if row_number == 1:
            hand += 1

        # Extract player information
        if start_hand == 1:
            match = re.search(pattern_dealer_position, line)
            match_player = re.search(pattern_player_information,line)
                
            if match:
                dealer_position = match.group(1)

            if match_player:
                seat_number = match_player.group(1)
                player_name = match_player.group(2)
                chips = match_player.group(3)

                # Assign the extracted details to the seat
                seats[seat_number] = {'player': player_name, 'chips': chips}
            
        
        # Print row number and whether it has content or not
        output_file.write(f"{row_number}: {has_content} {line}\n")
        #print(hand,start_hand, preflop, flop, turn, river, showdown)
        print(row_number, dealer_position, seats)

        

1 0 {}
2 1 {}
3 1 {'1': {'player': 'slambert666', 'chips': '500'}}
4 1 {'1': {'player': 'slambert666', 'chips': '500'}, '2': {'player': 'Dinahere', 'chips': '500'}}
5 1 {'1': {'player': 'slambert666', 'chips': '500'}, '2': {'player': 'Dinahere', 'chips': '500'}, '3': {'player': 'Mahu2021', 'chips': '500'}}
6 1 {'1': {'player': 'slambert666', 'chips': '500'}, '2': {'player': 'Dinahere', 'chips': '500'}, '3': {'player': 'Mahu2021', 'chips': '500'}}
7 1 {'1': {'player': 'slambert666', 'chips': '500'}, '2': {'player': 'Dinahere', 'chips': '500'}, '3': {'player': 'Mahu2021', 'chips': '500'}}
8 1 {'1': {'player': 'slambert666', 'chips': '500'}, '2': {'player': 'Dinahere', 'chips': '500'}, '3': {'player': 'Mahu2021', 'chips': '500'}}
9 1 {'1': {'player': 'slambert666', 'chips': '500'}, '2': {'player': 'Dinahere', 'chips': '500'}, '3': {'player': 'Mahu2021', 'chips': '500'}}
10 1 {'1': {'player': 'slambert666', 'chips': '500'}, '2': {'player': 'Dinahere', 'chips': '500'}, '3': {'player': 'Mahu